In [174]:
import dataclasses
from collections import Counter
from typing import Tuple, List, Dict, Union, Set, Generator, Iterator
from enum import Enum, auto
import time

from protein import trypsin
from measurement import read_mgf, PeptideMeasurement
from pyteomics import mass
from common import LYS, BSA


@dataclasses.dataclass
class AminoAcid:
    name: str
    # TODO: [i].mass should include C modifications
    mass: float


@dataclasses.dataclass
class Mod:
    description: str
    mass: float

    def __hash__(self):
        return (self.description, self.mass).__hash__()


@dataclasses.dataclass
class Modification:
    name: str
    mass: float
    count: int


@dataclasses.dataclass
class Peptide:
    beginning: int
    end: int
    seq: str
    amino_acids: List[AminoAcid]
    modifications: Dict[str, Tuple[Mod, int]]

    _aas = None
    _mass = None
    _minmass = None
    _maxmass = None

    def __init__(
        self,
        beginning: int,
        end: int,
        seq: str,
        modifications: Dict[str, Tuple[Mod, int]],
    ):
        self.beginning = beginning
        self.end = end
        self.seq = seq
        h2o = mass.calculate_mass(formula="H2O")
        self.amino_acids = [
            AminoAcid(aa, mass.calculate_mass(sequence=aa) - h2o) for aa in seq
        ]
        self._modifications = modifications

    def count(self, amino_acid):
        if self._aas is None:
            self._aas = Counter(self.seq)
        return self._aas[amino_acid]

    @property
    def zwitterion_mass(self):
        if self._mass is None:
            self._mass = mass.calculate_mass(
                sequence=self.seq, ion_type="M", charge=0
            ) - mass.calculate_mass(formula="H2O")
        return self._mass

    @property
    def min_mass(self):
        if self._minmass is None:
            neg = sum(
                m.mass * count for m, count in self.modifications_anywhere if m.mass < 0
            )
            self._minmass = self.zwitterion_mass + neg
        return self._minmass

    @property
    def max_mass(self):
        if self._maxmass is None:
            pos = sum(
                m.mass * count for m, count in self.modifications_anywhere if m.mass > 0
            )
            self._maxmass = self.zwitterion_mass + pos
        return self._maxmass

    def __getitem__(self, index: int):
        if self.beginning <= index < self.end:
            return self.amino_acids[index - self.beginning]
        return None

    @property
    def modifications_anywhere(self) -> Iterator[Tuple[Mod, int]]:
        return (x for x in self._modifications.values())


In [2]:
measurements = {m.scan: m for m in read_mgf("../data/mgf/190318_LYS_AT_50x_05.mgf")}

In [177]:
peptides = []
for b, e in trypsin(LYS):
    seq = LYS[b:e]
    met_ox = Mod("met_ox", 15.9949), sum(aa == "M" for aa in seq)
    peptides.append(Peptide(b, e, seq, {"M": met_ox}))

peptides

AttributeError: 'Peptide' object has no attribute 'modifications'

In [213]:

def within_bounds(reference_mass, measured_mass, ppm_error=10):
    return abs(reference_mass - measured_mass) <= err_margin(reference_mass, ppm_error)


def err_margin(reference_mass, ppm_error=10):
    return (ppm_error / 1e6) * reference_mass


def compute_error(reference_mass, measured_mass):
    return 1e6 * abs(measured_mass - reference_mass) / reference_mass


class State(Enum):
    BEFORE = auto()
    DURING = auto()


def set_tuple(t, i, x):
    return t[:i] + (x,) + t[i + 1 :]


# Pass None when you want to allow to skip a mod
def combine_modifications_2(
    modifications: List[List[Union[Mod, None]]],
    starting_mass: float,
    target_mass: float,
    ppm_error: float = 10,
) -> List[List[Mod]]:
    result = []

    def go(i, current, selection):
        if i == len(modifications):
            if within_bounds(current, target_mass, ppm_error):
                result.append(selection)
        else:
            for m in modifications[i]:
                if m is None:
                    go(i + 1, current, selection)
                else:
                    go(i + 1, current + m.mass, selection + (m,))

    go(0, current=starting_mass, selection=())
    return list(set(result))


# peptide_masses should be digestion peptides with H2O loss
# TODO: Přepsat na čitelnější verzi
def precursor_mass_matches(
    peptides: List[Peptide],
    measurement: PeptideMeasurement,
    alkylation_mass: float,
    max_inter_bonds: int,
    ppm_error: int = 10,
) -> List[str]:
    target = measurement.peptide_mass_estimate
    h2o = mass.calculate_mass(formula="H2O")
    h2 = mass.calculate_mass(formula="H2")

    result = []

    def go(
        i: int,
        current: float,
        min_raw_mass: float,
        max_raw_mass: float,
        selection: Tuple[int, ...],
        state: State,
        inter_bonds_left: int,
        cysteines_before: int,
        cysteines_now: int,
    ) -> None:
        internal_cysteines = cysteines_before + cysteines_now

        # Non-bonded cysteines are alkylated, or modified in another way
        if cysteines_now >= 0:
            max_posibble_mass = max_raw_mass + alkylation_mass * internal_cysteines
            upper_bound = max_posibble_mass + err_margin(max_posibble_mass, ppm_error)

            if target <= upper_bound:
                has_alkylated_cys = internal_cysteines % 2 == 1
                min_possible_mass = min_raw_mass + alkylation_mass * has_alkylated_cys
                lower_bound = min_possible_mass - err_margin(
                    min_possible_mass, ppm_error
                )

                if lower_bound <= target:
                    ranges = list(zip(selection[::2], (selection + (i,))[1::2]))

                    possible_mods: List[List[Mod]] = []

                    for b, e in ranges:
                        for p in peptides[b:e]:
                            for m, count in p.modifications_anywhere:
                                possible_mods += [
                                    [Mod(m.description, m.mass), None]
                                ] * count

                    max_intra_bonds = internal_cysteines // 2
                    for _ in range(max_intra_bonds):
                        possible_mods.append(
                            [Mod("cys_pair_alk", alkylation_mass * 2), None]
                        )

                    seq = "+".join(
                        "".join(p.seq for p in peptides[b:e]) for b, e in ranges
                    )

                    if has_alkylated_cys:
                        # One Cys has to be alkylated, as it can't be in a bond
                        possible_mods.append([Mod("cys_alk", alkylation_mass)])

                    combinations = combine_modifications_2(
                        possible_mods,
                        starting_mass=current,
                        target_mass=target,
                        ppm_error=ppm_error,
                    )

                    for modifications in combinations:
                        total_mass = current + sum(m.mass for m in modifications)

                        alkylated_pairs = sum(
                            m.description == "cys_pair_alk" for m in modifications
                        )
                        intra_bonds = max_intra_bonds - alkylated_pairs
                        inter_bonds = max_inter_bonds - inter_bonds_left

                        result.append(
                            {
                                "sequence": seq,
                                "cysteine_bonds": intra_bonds + inter_bonds,
                                "inter_bonds": inter_bonds,
                                "intra_bonds": intra_bonds,
                                "mass": total_mass,
                                "error": compute_error(total_mass, target),
                                "mods": modifications,
                            }
                        )

        if (
            i == len(peptides)
            or min_raw_mass - err_margin(min_raw_mass, ppm_error) > target
        ):
            # Either we're out of peptides to add
            # Or we're too high and we'll never correct it
            return
        else:
            if state == State.BEFORE:
                # Don't start yet
                go(
                    i + 1,
                    current,
                    min_raw_mass,
                    max_raw_mass,
                    selection,
                    state.BEFORE,
                    inter_bonds_left,
                    cysteines_before,
                    cysteines_now,
                )
            elif (
                state == State.DURING and min(inter_bonds_left, internal_cysteines) > 0
            ):
                # End this run, begin next one
                go(
                    i,
                    current + h2o - h2,
                    min_raw_mass + h2o - h2,
                    max_raw_mass + h2o - h2,
                    selection + (i,),
                    state.BEFORE,
                    inter_bonds_left - 1,
                    internal_cysteines - 1,
                    -1,
                )

            # Take this one, and either begin or continue this run
            go(
                i + 1,
                current + peptides[i].zwitterion_mass,
                min_raw_mass + peptides[i].min_mass,
                max_raw_mass + peptides[i].max_mass,
                selection + (i,) if state == State.BEFORE else selection,
                State.DURING,
                inter_bonds_left,
                cysteines_before,
                cysteines_now + peptides[i].count("C"),
            )

    go(
        0,
        current=h2o,
        min_raw_mass=h2o,
        max_raw_mass=h2o,
        selection=(),
        state=State.BEFORE,
        inter_bonds_left=max_inter_bonds,
        cysteines_before=0,
        cysteines_now=0,
    )

    return result


In [576]:
import csv

FILE_PATH = "../out/precursor_matches_bsa_at_2_bonds.csv"

start_time = time.time()

with open(FILE_PATH, "w") as f:
    field_names = ["scan", "sequence", "mass", "error", "cysteine_mods", "mods"]
    writer = csv.DictWriter(f, fieldnames=field_names)
    writer.writeheader()

    for scan, measurement in measurements.items():
        for match in precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ):
            writer.writerow({"scan": scan} | match)
end_time = time.time()

print(f"This takes {end_time - start_time} seconds")

This takes 2095.511372089386 seconds


In [216]:
start_time = time.time()

for scan, measurement in measurements.items():
    for match in sorted(
        precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ),
        key=lambda m: m["sequence"],
    ):
        print(f"{scan}: {match}")

end_time = time.time()
print(f"This takes {end_time - start_time} seconds")

845: {'sequence': 'CELAAAMK+GCR', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1183.51476996307, 'error': 0.17497809442811452, 'mods': (Mod(description='met_ox', mass=15.9949),)}
846: {'sequence': 'RHGLDNYR', 'cysteine_bonds': 0, 'inter_bonds': 0, 'intra_bonds': 0, 'mass': 1029.51042528457, 'error': 0.046330920919352085, 'mods': ()}
848: {'sequence': 'CELAAAMK+GCR', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1183.51476996307, 'error': 0.7012779318072832, 'mods': (Mod(description='met_ox', mass=15.9949),)}
849: {'sequence': 'CELAAAMK+CK', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1098.4871582329, 'error': 0.45653075413820915, 'mods': (Mod(description='met_ox', mass=15.9949),)}
852: {'sequence': 'CELAAAMKR+CK', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1238.5933692565, 'error': 0.363565438838716, 'mods': ()}
854: {'sequence': 'CELAAAMKR+GCR', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'm

In [214]:
# 7012, 7013

for sc in [5205, 4956, 5408, 5545, 5564]:
    print(sc)

    matches = precursor_mass_matches(
        peptides,
        measurements[sc],
        alkylation_mass=57.0214,
        max_inter_bonds=2,
        ppm_error=15,
    )
    for m in matches:
        print(m)

5205
4956
{'sequence': 'KVFGRCELAAAMKRHGLDNYR+TPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWR', 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 7165.52511386176, 'error': 14.774213611373956, 'mods': (Mod(description='cys_pair_alk', mass=114.0428),)}
5408
5545
5564
{'sequence': 'KVFGRCELAAAMKR+WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKK+CKGTDVQAWIRGCR', 'cysteine_bonds': 3, 'inter_bonds': 2, 'intra_bonds': 1, 'mass': 7103.43762553123, 'error': 1.7778985190698375, 'mods': (Mod(description='cys_alk', mass=57.0214),)}


In [215]:
start_time = time.time()

for scan, measurement in measurements.items():
    for match in sorted(
        precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ),
        key=lambda m: m["sequence"],
    ):
        seq = match["sequence"]
        mods = match["mods"]
        print(f"{scan}: {seq}")

end_time = time.time()
print(f"This takes {end_time - start_time} seconds")

845: CELAAAMK+GCR
846: RHGLDNYR
848: CELAAAMK+GCR
849: CELAAAMK+CK
852: CELAAAMKR+CK
854: CELAAAMKR+GCR
858: NRCK+GCR
889: CELAAAMK+CK
890: CELAAAMKR+GCR
892: CELAAAMK+CK
893: CELAAAMKR
914: CELAAAMKR
917: HGLDNYR
956: CELAAAMK+GCR
957: CELAAAMK+GCR
964: CELAAAMK+GCR
974: GYSLGNWVCAAK
1007: CELAAAMKR+GCR
1015: HGLDNYR
1033: CELAAAMK+GCR
1036: CELAAAMK+GCR
1038: CELAAAMK+GCR
1057: CELAAAMKR+GCRL
1070: HGLDNYR
1072: CELAAAMKR+GCRL
1086: CELAAAMK+GCR
1093: CELAAAMKR
1097: CELAAAMK
1100: CELAAAMKR
1109: CELAAAMKR+GCRL
1122: HGLDNYR
1128: CELAAAMKR
1138: CELAAAMK+GCR
1157: CELAAAMK
1162: CELAAAMKR+GCRL
1169: CELAAAMKR+GCRL
1171: CELAAAMKR+GCRL
1187: HGLDNYR
1200: CELAAAMKR
1207: CELAAAMK+GCR
1216: CELAAAMK+NRCK
1223: CELAAAMK+GCRL
1224: CELAAAMK
1242: CELAAAMKR+GCRL
1259: HGLDNYR
1265: CELAAAMK+GCRL
1269: CELAAAMKR
1276: CELAAAMK+GCR
1281: CELAAAMKR+GCRL
1287: CELAAAMK+GCRL
1296: CELAAAMK
1303: WWCNDGR+CK
1310: CELAAAMKR+GCRL
1324: WWCNDGR+CK
1332: CELAAAMKR
1335: HGLDNYR
1336: CELAAAMK+GCR

In [ ]:

@dataclasses.dataclass
class Run:
    beginning: int
    end: int
    forward: int

    def __init__(self, beggining: int, end: int):
        self.beginning = beggining
        self.end = end
        self.forward = 1 if end > beggining else -1


class MultiPeptide:
    _peptides: List[Peptide]
    _cysteine_bonds: List[Tuple[int, int]]
    _ranges: List[Tuple[int, int]]
    _modifications: Dict[str, Tuple[Mod, int]]

    def __getitem__(self, index: int) -> AminoAcid:
        for p in self._peptides:
            aa = p[index]
            if aa is not None:
                return aa

    def bond_partner(self, index: int) -> Union[int, None]:
        for x, y in self._cysteine_bonds:
            if index == x:
                return y
            if index == y:
                return x
        return None

    def __contains__(self, index: int):
        return any(index in range(x, y) for x, y in self._ranges)

    def is_break(self, index: int) -> bool:
        return any(index in range(x + 1, y - 1) for x, y in self._ranges)

    def run_from(self, index: int, forward: bool) -> Run:
        for x, y in self._ranges:
            if x <= index <= y:
                if forward:
                    return Run(index, y)
                else:
                    return Run(index, x - 1)

    @property
    def modded_amino_acids(self) -> Set[str]:
        return {mod.target_aa for mod in self._modifications}

    def count(self, amino_acid: str):
        return sum(p.count(amino_acid) for p in self._peptides)

    def modifications_on(self, amino_acid: str) -> Tuple[Mod, int]:
        return self._modifications[amino_acid]

    def run_mass(self, run: Run) -> float:
        for p in self._peptides:
            if p.beginning == run.beginning and run.end == p.end:
                return p.zwitterion_mass
            elif p.beginning <= run.beginning and run.end <= p.end:
                # MAYBE: Cache this
                return sum(
                    p[i] for i in range(run.beginning, run.end, step=run.forward)
                )


# TODO: Add min_mass counting
def match_fragments(target, peptide: MultiPeptide, breaks, ppm_error=10):

    result = []
    b_ion_mod = -mass.calculate_mass(formula="OH")
    y_ion_mod = 0

    h2o = mass.calculate_mass(formula="H2O")
    nh3 = mass.calculate_mass(formula="NH3")
    sulphur = mass.calculate_mass(formula="S")
    h2 = mass.calculate_mass(formula="H2")

    def go(
        i,
        current,
        runs: Tuple[Run, ...],
        selection: Tuple[int, ...],
        breaks_left: int,
        broken_cysteines: Tuple[int, ...],
        neutral_losses_count: int,
        modded_amino_acids: Dict[str, int],
    ):
        if breaks_left == 0:
            sum_rest = sum(peptide.run_mass(run) for run in runs)
            go(
                i,
                current + sum_rest,
                (),
                selection + tuple(n for s in runs for n in [s.beginning, s.end]),
                0,
                broken_cysteines,
                neutral_losses_count,  # MAYBE: Add more neutral losses?
                ...,  # TODO Sum rest of modded aas
            )

        if within_bounds(current, target, ppm_error):
            ranges = [
                (x, y) if y > x else (y, x)
                for x, y in zip(selection[::2], (selection + (i,))[1::2])
            ]

            mod_array = []

            final_mass = current

            for aa, count in modded_amino_acids:
                peptide_mod, peptide_mod_count = peptide.modifications_on(aa)

                minimum_mods = max(peptide_mod_count - (peptide.count(aa) - count), 0)
                final_mass += minimum_mods * peptide_mod

                # How many can I have
                maximum_mods = min(peptide_mod_count, count)
                # Optional mods
                for _ in range(maximum_mods - minimum_mods):
                    mod_array.append([(None, 0), peptide_mod])

            for _ in range(neutral_losses_count):
                # MAYBE: Make this more granular?
                mod_array += [
                    ("–H2O neutrall loss", -h2o),
                    ("–NH3 neutrall loss", -nh3),
                ]

            for c in broken_cysteines:
                symmetric = (
                    j := peptide.bond_partner(c) is not None
                ) and j in broken_cysteines

                # Symmetry breaking
                if symmetric and i > j:
                    continue

                if symmetric:
                    mod_array.append([("-SSH + / or -SH + =S", -h2)])
                else:
                    mod_array.append(
                        [
                            ("-SSH", sulphur),
                            ("- /", -(sulphur + h2)),
                            ("=S", -h2),
                            ("-SH", 0),
                        ]
                    )

            combinations = combine_modifications_2(
                mod_array,
                starting_mass=final_mass,
                target_mass=target,
                ppm_error=ppm_error,
            )

            for modifications in combinations:
                total_mass = final_mass + sum(m.mass for m in modifications)
                result.append(
                    {
                        "ranges": ranges,
                        "mass": total_mass,
                        "error": compute_error(total_mass, target),
                        "mods": c,
                    }
                )

        # Nowhere to go next
        if len(runs) == 0 or len(runs) == 1 and i == runs[0].end:
            return

        if i == runs[0].beginning:
            selection += (i,)

        # We have to end this run
        if i == runs[0].end:
            go(
                runs[1].beginning,
                current,
                runs[1:],
                selection + (i,),
                breaks_left,
                broken_cysteines,
                neutral_losses_count,
                modded_amino_acids,
            )
        else:
            aa = peptide[i].name
            if aa == "C" and (j := peptide.bond_partner(i) is not None):

                if i < j:
                    # Make sure we don't cross over
                    next_runs = (
                        peptide.run_from(j - 1, forward=False),
                        peptide.run_from(j + 1, forward=True),
                    )

                    # Continue, don't break the SS bond
                    go(
                        i + runs[0].forward,
                        current + peptide[i].mass + peptide[j].mass,
                        runs[1:] + next_runs,
                        selection + (j, j + 1),
                        breaks_left,
                        broken_cysteines,
                        neutral_losses_count,
                        modded_amino_acids | {aa: modded_amino_acids[aa] + 1}
                        if aa in peptide.modded_amino_acids
                        else {},
                    )

                # Continue, break the SS bond
                go(
                    i + runs[0].forward,
                    current + peptide[i].mass,
                    runs[1:],
                    selection,
                    breaks_left - 1,
                    broken_cysteines + (i,),
                    neutral_losses_count,
                    modded_amino_acids | {aa: modded_amino_acids[aa] + 1}
                    if aa in peptide.modded_amino_acids
                    else {},
                )
            else:
                # Go forward, adding another amino acid
                go(
                    i + runs[0].forward,
                    current + peptide[i].mass,
                    runs,
                    selection,
                    breaks_left,
                    broken_cysteines,
                    neutral_losses_count,
                    modded_amino_acids | {aa: modded_amino_acids[aa] + 1}
                    if aa in peptide.modded_amino_acids
                    else {},
                )

            # Break the peptide bond, start next run
            go(
                runs[1].beginning,
                current + b_ion_mod if runs[0].forward > 0 else y_ion_mod,
                runs[1:],
                selection + (i,),
                breaks_left - 1,
                broken_cysteines,
                neutral_losses_count + 1,
                modded_amino_acids,
            )

    # TODO: Optimize so that we aren't too close to the end
    for beginning in peptide:
        modifier = 0
        if peptide.is_break(beginning):
            breaks -= 1
            modifier = y_ion_mod

        go(
            beginning,
            current=0 + modifier,
            runs=(peptide.run_from(beginning, forward=True),),
            selection=(),
            breaks_left=breaks,
            broken_cysteines=(),
            neutral_losses_count=0,
            modded_amino_acids={aa: 0 for aa in peptide.modded_amino_acids},
        )

    return result


# TODO: Target should be with fixed charge (i.e. try all charges in a loop)
# TODO: Peptide should have fixed cysteine bonds (i.e. try all cysteine combinations in a loop)